# Tutorial 1: Part 2

Objectives:
- Learn how to define a simple lattice and compute the TWISS functions using MAD-X.
- Thick vs thin lense approximation TWISS comparison for a lattice with only quadrupoles.
- Tune and $\beta$-function dependence on K1.

**My first accelerator: a FODO cell**

1. Make a simple lattice cell with $L_{cell}$= 100 m made of focusing and defocusing qudrupoles (so called FODO cell). Each quadrupole is 5 m long ($L_{quad}$). Put the start of the first qudrupole at the start of the sequence. Each quadrupole has a focal length f=200 m (k1 x $L_{quad}$ = 1/f in thin lense approximation).


2. Define a proton beam at $E_{tot}$= 2 GeV. Activate the sequence and try to find the periodic solution and plot the $\beta$-functions. If you found $\beta_{max}$= 460 m you succeded.


3. Using the plot you obtained can you estimate the phase advance of the cell. Compare with the tunes obtained with the TWISS.


4. Try with $E_{tot}$ = 0.7 GeV: what is the MAD-X error message?


5. Try with f= 20 m: what is the MAD-x error message?

**Matching the FODO cell using a parametric plot**

6. Try to twiss it powering the quds to obtain a $\Delta \mu \approx 90^\circ$ in the cell using the thin lense approximation (using the figures from Tutorial 1: Part 1). What is the actual phase advance computed by MAD-X?


7. What is the $\beta_{max}$? Compare with the thin lense approximation (using the figures from Tutorial 1: Part 1).


8. Compute the maximum beam size $\sigma$ assuming a normalized emittance of 3 mrad mm and $E_{tot}= 7 TeV$.


9. Halve the focusing strength of the quadrupoles, what is the effect of it on the $\beta_{max}$ and $\beta_{min}$ and on the $\Delta \mu$? Compare with the parametric plots.

In [1]:
#Import the needed libraries
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from cpymad.madx import Madx 

# Launching MAD-X

In [2]:
myMad = Madx(stdout=True)

1. Make a simple lattice cell with 𝐿𝑐𝑒𝑙𝑙= 100 m made of focusing and defocusing qudrupoles (so called FODO cell). Each quad is 5 m long (𝐿𝑞𝑢𝑎𝑑). Put the start of the first qudrupole at the start of the sequence. Each quadrupole has a focal length f=200 m (k1 x 𝐿𝑞𝑢𝑎𝑑 = 1/f in thin lense approximation).


2. Define a proton beam at $E_{tot}$= 2 GeV. Activate the sequence and try to find the periodic solution and plot the $\beta$-functions. If you found $\beta_{max} \approx$ 460 m you succeded.

In [99]:
myString='''
! *********************************************************************
! Definition of parameters
! *********************************************************************

l_cell=??;
quadrupoleLenght=??;
f=??;
myK:=1/f/quadrupoleLenght;// m^-2

! *********************************************************************
! Definition of magnets
! ********************************************************************* 
QF: quadrupole, L=??, K1:=??;
QD: quadrupole, L=??, K1:=??;

! *********************************************************************
! Definition of sequence
! *********************************************************************
myCell:sequence, refer=entry, L=L_CELL;
quadrupole1: QF, at=??;
quadrupole2: QD, at=??;
endsequence;

! *********************************************************************
! Definition of beam
! *********************************************************************
beam, particle=proton, energy=??;

! *********************************************************************
! Use of the sequence
! *********************************************************************
use, sequence=myCell;

! *********************************************************************
! TWISS
! *********************************************************************
title, 'My first twiss';
twiss, file=MyfirstFODO.madx;
plot, haxis=s, vaxis=betx,bety,dx,colour=100, title="test",file=MyfirstFODO;
'''
myMad.input(myString);

In [100]:
#############################################################
# Extract information from SUMM table using MADX commands   #
#############################################################
myString='''
value, table(SUMM,Q1);
value, table(SUMM,betymax);
'''
myMad.input(myString);

And for the vertical plane?

In [ ]:
#############################################################
# Extract information from SUMM table using python commands #
#############################################################
# SUMM table
myDF=myMad.table.summ.dframe()
myDF
myDF["q1"]

In [ ]:
##############################################################
# Extract information from TWISS table using python commands #
##############################################################
# TWISS table
myDF=myMad.table.twiss.dframe()
myDF[['name','s','betx','bety','alfx','alfy','mux','muy']]

In [ ]:
############################
# Plot with python example #
############################
%matplotlib notebook
plt.plot(myDF['s'],myDF['betx'],'.-b',label='$\\beta_x$')
plt.plot(myDF['s'],myDF['bety'],'.-r',label='$\\beta_y$')
plt.legend()
plt.grid()
plt.xlabel('s [m]')
plt.ylabel('[m]')
plt.title('My first FODO cell')

If you found $\beta_{max}$= 463.6 m you succeded!

# Tune from the plot
3. Using the plot you obtained can you estimate the phase advance of the cell. Compare with the tunes obtained with the TWISS.

Remember that the unit of phase in MAD-X is [2$\pi$].

In [12]:
# Phase advance from the MAD-X TWISS

# Varying the energy
4. Try with $E_{tot}$ = 0.7 GeV: what is the MAD-X error message? 

In [ ]:
# with this simple wrapper in case of error the code enter in an infinite loop you have to stop manually.
myString='''
beam, particle=proton, energy=??;
title, 'My third twiss';
twiss;
'''
myMad.input(myString);

# With f=20 m
5. Try with f= 20 m: what is the MAD-x error message?

In [108]:
myMad = Madx(stdout=True)

In [109]:
myString='''
! *********************************************************************
! Definition of parameters
! *********************************************************************

l_cell=??;
quadrupoleLenght=??;
f=??;
myK:=1/f/quadrupoleLenght;// m^-2

! *********************************************************************
! Definition of magnet
! ********************************************************************* 
QF: quadrupole, L=??, K1:=??;
QD: quadrupole, L=??, K1:=??;


! *********************************************************************
! Definition of sequence
! *********************************************************************
myCell:sequence, refer=entry, L=L_CELL;
quadrupole1: QF, at=??;
quadrupole2: QD, at=??;
endsequence;

! *********************************************************************
! Definition of beam
! *********************************************************************
beam, particle=proton, energy=??;

! *********************************************************************
! Use of the sequence
! *********************************************************************
use, sequence=myCell;

! *********************************************************************
! TWISS
! *********************************************************************
select, flag=twiss, clear;
select, flag=twiss, column=name, keyword, s, betx, alfx, mux, bety, alfy, muy, x, px, y, py, dx, dy, dx, dpx, dy, dpy;
twiss, file=f20.txt;
'''
myMad.input(myString);

# **Matching the FODO cell using a parametric plot**
6. Try to twiss it powering the quadrupoles to obtain a $\Delta \mu \approx 90^\circ$ in the cell using the thin lense approximation (using the figures from Tutorial 1: Part 1). What is the actual phase advance computed by MAD-X?

In [110]:
myMad = Madx(stdout=True)

In [3]:
# From the plot from Tutoria 1 - part1, for 90 degrees phase advance

In [112]:
myString='''
! Definition of parameters
quadrupoleLenght=??;
cellLength=??;
myK=??/cellLength/quadrupoleLenght;// m^-2

! Definition of magnet
QF: quadrupole, L=quadrupoleLenght, K1:=??;
QD: quadrupole, L=quadrupoleLenght, K1:=??;

! Definition of sequence
myCell:sequence, refer=entry, L=cellLength;
q1: QF, at=??;
q2: QD, at=??;
endsequence;

! Definition of beam
beam, particle=proton, energy=??;
use, sequence=myCell;

! TWISS
select, flag=twiss, clear;
select, flag=twiss, column=name, keyword, s, betx, alfx, mux, bety, alfy, muy;
twiss;
value, myK;
'''
myMad.input(myString);

In [6]:
##############################################################
# Extract information from TWISS table using python commands #
##############################################################


In [114]:
##############################################################
# Extract information from SUMM table using MADX commands    #
##############################################################

In [7]:
#Phase advance computed by MADX in rad


In [8]:
#Phase advance computed by MADX in degrees


7. What is the $\beta_{max}$? Compare with the thin lense approximation (using the figures from Tutorial 1: Part 1).

In [4]:
# From the MAD-X calculation


In [5]:
#From the parametric plot Figure 2 


8. Compute the maximum beam size $\sigma$ assuming a normalized emittance of 3 mrad mm and $E_{tot}= 7 TeV$.


9. Halve the focusing strength of the quadrupoles, what is the effect of it on the $\beta_{max}$ and $\beta_{min}$ and on the $\Delta \mu$? Compare with the parametric plots.

In [122]:
myString='''
myK=??/cellLength/quadrupoleLenght;// m^-2
twiss, file=firstTwiss.txt;
'''
myMad.input(myString);

In [123]:
#####################################################################
# Extract information from SUMM table using python or MADX commands #
#####################################################################

In [9]:
# bmax from parametric plots


In [13]:
# Value from MADX
